# Feature Engineering en SQL

A continuación, veremos cómo calcular diferentes variables para el feature engineering utilizando SQL.


In [1]:
import duckdb
import pandas as pd
from sqlalchemy import create_engine

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///../../../buckets/b1/db/dmeyf.db

In [2]:
base_path = '/home/fedepicado/'
modelos_path = base_path + 'buckets/b1/modelos/'
db_path = base_path + 'buckets/b1/db/'
dataset_path = base_path + 'buckets/b1/datasets/'
exp_path = base_path + 'buckets/b1/exp/'
dataset_file = 'competencia_03_DQ_agrupacion_var.parquet'
full_path = dataset_path + dataset_file


In [3]:
%%sql
ROLLBACK;

,Success


In [4]:
%%sql
create or replace table competencia_03_DQ_agrupacion_var as
select
    *
from read_parquet('../../../buckets/b1/datasets/competencia_03_DQ_agrupacion_var.parquet')

,Success


In [5]:
df=pd.read_parquet(full_path)

In [6]:
df.shape

(4901237, 149)

In [7]:
campos=df.columns.to_list()
print(campos[-1])
campos.pop(-1) #elimino clase ternaria
print(campos[-1])

clase_ternaria
Visa_mpagominimo


In [8]:
%%sql
CREATE OR REPLACE MACRO suma_sin_null(a, b) AS ifnull(a, 0) + ifnull(b, 0)

,Success


In [9]:
%%sql
CREATE OR REPLACE MACRO div_sin_cero(a, b) as
    case
        when ifnull(b, 0) = 0 THEN NULL
        else ifnull(a, 0) / ifnull(b, 1)
    end

,Success


In [1]:
var_monetarias= ["mrentabilidad","mrentabilidad_annual","mcomisiones","mactivos_margen","mpasivos_margen","mcuenta_corriente",
               "mcaja_ahorro","mcaja_ahorro_adicional","mcaja_ahorro_dolares","mcuentas_saldo","mautoservicio","mprestamos_prendarios",
                 "mprestamos_hipotecarios","mplazo_fijo_dolares",
               "mplazo_fijo_pesos","minversion1_pesos","minversion1_dolares","minversion2","mpayroll","mpayroll2","mcuenta_debitos_automaticos",
               "mttarjeta_visa_debitos_automaticos","mttarjeta_master_debitos_automaticos","mpagomiscuentas","mcajeros_propios_descuentos",
               "mtarjeta_visa_descuentos","mtarjeta_master_descuentos","mcomisiones_mantenimiento","mcomisiones_otras","mforex_buy",
               "mtransferencias_recibidas","mtransferencias_emitidas","mextraccion_autoservicio","mcheques_depositados","mcheques_emitidos",
               "mcheques_depositados_rechazados","mcheques_emitidos_rechazados","matm","matm_other","mfinanciacion_limite_total", 
                "msaldototal_total", "msaldopesos_total","msaldodolares_total", "mconsumospesos_total",
                "mconsumosdolares_total", "mlimitecompra_total", "madelantopesos_total", "madelantodolares_total", 
                "mpagado_total", "mpagospesos_total", "mpagosdolares_total", "mconsumototal_total", 
                "mpagominimo_total", "mtarjetas_consumo"]

## Lag delta r_diff

In [2]:
nuevos_campos = []

for campo in var_monetarias:
    nuevos_campos.append(f", {campo} - LAG({campo}, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_{campo}")
    nuevos_campos.append(f", AVG({campo}) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS {campo}_media_5")

nuevos_campos_str = "\n".join(nuevos_campos)

dif2_campos = []

for campo in var_monetarias:
    dif2_campos.append(f", dif1_{campo} - LAG(dif1_{campo}, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_{campo}")

dif2_campos_str = "\n".join(dif2_campos)

lags_campos = []

for campo in var_monetarias:
    lags_campos.append(f", LAG({campo}, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_{campo}")
    lags_campos.append(f", LAG({campo}, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_{campo}")

lags_campos_str = "\n".join(lags_campos)

consulta_sql = f"""
CREATE OR REPLACE TABLE competencia_03_DQ_ft AS
WITH cte_dif1 AS (
    SELECT
        *
        {nuevos_campos_str}
        {lags_campos_str}
    FROM competencia_03_DQ_agrupacion_var
),
cte_dif2 AS (
    SELECT
        *
        {dif2_campos_str}
    FROM cte_dif1
)
SELECT *
FROM cte_dif2;
"""

print(consulta_sql)




CREATE OR REPLACE TABLE competencia_03_DQ_ft AS
WITH cte_dif1 AS (
    SELECT
        *
        , mrentabilidad - LAG(mrentabilidad, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mrentabilidad
, AVG(mrentabilidad) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mrentabilidad_media_5
, mrentabilidad_annual - LAG(mrentabilidad_annual, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mrentabilidad_annual
, AVG(mrentabilidad_annual) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mrentabilidad_annual_media_5
, mcomisiones - LAG(mcomisiones, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mcomisiones
, AVG(mcomisiones) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mcomisiones_media_5
, mactivos_margen - LAG(mactivos_margen, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_

In [ ]:
%%sql
CREATE OR REPLACE TABLE competencia_03_DQ_ft AS
WITH cte_dif1 AS (
    SELECT
        *
        , mrentabilidad - LAG(mrentabilidad, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mrentabilidad
, AVG(mrentabilidad) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mrentabilidad_media_5
, mrentabilidad_annual - LAG(mrentabilidad_annual, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mrentabilidad_annual
, AVG(mrentabilidad_annual) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mrentabilidad_annual_media_5
, mcomisiones - LAG(mcomisiones, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mcomisiones
, AVG(mcomisiones) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mcomisiones_media_5
, mactivos_margen - LAG(mactivos_margen, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mactivos_margen
, AVG(mactivos_margen) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mactivos_margen_media_5
, mpasivos_margen - LAG(mpasivos_margen, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mpasivos_margen
, AVG(mpasivos_margen) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mpasivos_margen_media_5
, mcuenta_corriente - LAG(mcuenta_corriente, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mcuenta_corriente
, AVG(mcuenta_corriente) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mcuenta_corriente_media_5
, mcaja_ahorro - LAG(mcaja_ahorro, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mcaja_ahorro
, AVG(mcaja_ahorro) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mcaja_ahorro_media_5
, mcaja_ahorro_adicional - LAG(mcaja_ahorro_adicional, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mcaja_ahorro_adicional
, AVG(mcaja_ahorro_adicional) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mcaja_ahorro_adicional_media_5
, mcaja_ahorro_dolares - LAG(mcaja_ahorro_dolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mcaja_ahorro_dolares
, AVG(mcaja_ahorro_dolares) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mcaja_ahorro_dolares_media_5
, mcuentas_saldo - LAG(mcuentas_saldo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mcuentas_saldo
, AVG(mcuentas_saldo) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mcuentas_saldo_media_5
, mautoservicio - LAG(mautoservicio, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mautoservicio
, AVG(mautoservicio) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mautoservicio_media_5
, mprestamos_prendarios - LAG(mprestamos_prendarios, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mprestamos_prendarios
, AVG(mprestamos_prendarios) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mprestamos_prendarios_media_5
, mprestamos_hipotecarios - LAG(mprestamos_hipotecarios, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mprestamos_hipotecarios
, AVG(mprestamos_hipotecarios) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mprestamos_hipotecarios_media_5
, mplazo_fijo_dolares - LAG(mplazo_fijo_dolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mplazo_fijo_dolares
, AVG(mplazo_fijo_dolares) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mplazo_fijo_dolares_media_5
, mplazo_fijo_pesos - LAG(mplazo_fijo_pesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mplazo_fijo_pesos
, AVG(mplazo_fijo_pesos) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mplazo_fijo_pesos_media_5
, minversion1_pesos - LAG(minversion1_pesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_minversion1_pesos
, AVG(minversion1_pesos) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS minversion1_pesos_media_5
, minversion1_dolares - LAG(minversion1_dolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_minversion1_dolares
, AVG(minversion1_dolares) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS minversion1_dolares_media_5
, minversion2 - LAG(minversion2, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_minversion2
, AVG(minversion2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS minversion2_media_5
, mpayroll - LAG(mpayroll, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mpayroll
, AVG(mpayroll) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mpayroll_media_5
, mpayroll2 - LAG(mpayroll2, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mpayroll2
, AVG(mpayroll2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mpayroll2_media_5
, mcuenta_debitos_automaticos - LAG(mcuenta_debitos_automaticos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mcuenta_debitos_automaticos
, AVG(mcuenta_debitos_automaticos) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mcuenta_debitos_automaticos_media_5
, mttarjeta_visa_debitos_automaticos - LAG(mttarjeta_visa_debitos_automaticos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mttarjeta_visa_debitos_automaticos
, AVG(mttarjeta_visa_debitos_automaticos) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mttarjeta_visa_debitos_automaticos_media_5
, mttarjeta_master_debitos_automaticos - LAG(mttarjeta_master_debitos_automaticos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mttarjeta_master_debitos_automaticos
, AVG(mttarjeta_master_debitos_automaticos) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mttarjeta_master_debitos_automaticos_media_5
, mpagomiscuentas - LAG(mpagomiscuentas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mpagomiscuentas
, AVG(mpagomiscuentas) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mpagomiscuentas_media_5
, mcajeros_propios_descuentos - LAG(mcajeros_propios_descuentos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mcajeros_propios_descuentos
, AVG(mcajeros_propios_descuentos) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mcajeros_propios_descuentos_media_5
, mtarjeta_visa_descuentos - LAG(mtarjeta_visa_descuentos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mtarjeta_visa_descuentos
, AVG(mtarjeta_visa_descuentos) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mtarjeta_visa_descuentos_media_5
, mtarjeta_master_descuentos - LAG(mtarjeta_master_descuentos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mtarjeta_master_descuentos
, AVG(mtarjeta_master_descuentos) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mtarjeta_master_descuentos_media_5
, mcomisiones_mantenimiento - LAG(mcomisiones_mantenimiento, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mcomisiones_mantenimiento
, AVG(mcomisiones_mantenimiento) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mcomisiones_mantenimiento_media_5
, mcomisiones_otras - LAG(mcomisiones_otras, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mcomisiones_otras
, AVG(mcomisiones_otras) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mcomisiones_otras_media_5
, mforex_buy - LAG(mforex_buy, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mforex_buy
, AVG(mforex_buy) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mforex_buy_media_5
, mtransferencias_recibidas - LAG(mtransferencias_recibidas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mtransferencias_recibidas
, AVG(mtransferencias_recibidas) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mtransferencias_recibidas_media_5
, mtransferencias_emitidas - LAG(mtransferencias_emitidas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mtransferencias_emitidas
, AVG(mtransferencias_emitidas) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mtransferencias_emitidas_media_5
, mextraccion_autoservicio - LAG(mextraccion_autoservicio, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mextraccion_autoservicio
, AVG(mextraccion_autoservicio) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mextraccion_autoservicio_media_5
, mcheques_depositados - LAG(mcheques_depositados, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mcheques_depositados
, AVG(mcheques_depositados) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mcheques_depositados_media_5
, mcheques_emitidos - LAG(mcheques_emitidos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mcheques_emitidos
, AVG(mcheques_emitidos) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mcheques_emitidos_media_5
, mcheques_depositados_rechazados - LAG(mcheques_depositados_rechazados, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mcheques_depositados_rechazados
, AVG(mcheques_depositados_rechazados) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mcheques_depositados_rechazados_media_5
, mcheques_emitidos_rechazados - LAG(mcheques_emitidos_rechazados, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mcheques_emitidos_rechazados
, AVG(mcheques_emitidos_rechazados) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mcheques_emitidos_rechazados_media_5
, matm - LAG(matm, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_matm
, AVG(matm) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS matm_media_5
, matm_other - LAG(matm_other, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_matm_other
, AVG(matm_other) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS matm_other_media_5
, mfinanciacion_limite_total - LAG(mfinanciacion_limite_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mfinanciacion_limite_total
, AVG(mfinanciacion_limite_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mfinanciacion_limite_total_media_5
, msaldototal_total - LAG(msaldototal_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_msaldototal_total
, AVG(msaldototal_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS msaldototal_total_media_5
, msaldopesos_total - LAG(msaldopesos_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_msaldopesos_total
, AVG(msaldopesos_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS msaldopesos_total_media_5
, msaldodolares_total - LAG(msaldodolares_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_msaldodolares_total
, AVG(msaldodolares_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS msaldodolares_total_media_5
, mconsumospesos_total - LAG(mconsumospesos_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mconsumospesos_total
, AVG(mconsumospesos_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mconsumospesos_total_media_5
, mconsumosdolares_total - LAG(mconsumosdolares_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mconsumosdolares_total
, AVG(mconsumosdolares_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mconsumosdolares_total_media_5
, mlimitecompra_total - LAG(mlimitecompra_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mlimitecompra_total
, AVG(mlimitecompra_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mlimitecompra_total_media_5
, madelantopesos_total - LAG(madelantopesos_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_madelantopesos_total
, AVG(madelantopesos_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS madelantopesos_total_media_5
, madelantodolares_total - LAG(madelantodolares_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_madelantodolares_total
, AVG(madelantodolares_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS madelantodolares_total_media_5
, mpagado_total - LAG(mpagado_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mpagado_total
, AVG(mpagado_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mpagado_total_media_5
, mpagospesos_total - LAG(mpagospesos_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mpagospesos_total
, AVG(mpagospesos_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mpagospesos_total_media_5
, mpagosdolares_total - LAG(mpagosdolares_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mpagosdolares_total
, AVG(mpagosdolares_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mpagosdolares_total_media_5
, mconsumototal_total - LAG(mconsumototal_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mconsumototal_total
, AVG(mconsumototal_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mconsumototal_total_media_5
, mpagominimo_total - LAG(mpagominimo_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mpagominimo_total
, AVG(mpagominimo_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mpagominimo_total_media_5
, mtarjetas_consumo - LAG(mtarjetas_consumo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mtarjetas_consumo
, AVG(mtarjetas_consumo) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mtarjetas_consumo_media_5
        , LAG(mrentabilidad, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mrentabilidad
, LAG(mrentabilidad, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mrentabilidad
, LAG(mrentabilidad_annual, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mrentabilidad_annual
, LAG(mrentabilidad_annual, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mrentabilidad_annual
, LAG(mcomisiones, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mcomisiones
, LAG(mcomisiones, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mcomisiones
, LAG(mactivos_margen, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mactivos_margen
, LAG(mactivos_margen, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mactivos_margen
, LAG(mpasivos_margen, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mpasivos_margen
, LAG(mpasivos_margen, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mpasivos_margen
, LAG(mcuenta_corriente, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mcuenta_corriente
, LAG(mcuenta_corriente, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mcuenta_corriente
, LAG(mcaja_ahorro, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mcaja_ahorro
, LAG(mcaja_ahorro, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mcaja_ahorro
, LAG(mcaja_ahorro_adicional, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mcaja_ahorro_adicional
, LAG(mcaja_ahorro_adicional, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mcaja_ahorro_adicional
, LAG(mcaja_ahorro_dolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mcaja_ahorro_dolares
, LAG(mcaja_ahorro_dolares, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mcaja_ahorro_dolares
, LAG(mcuentas_saldo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mcuentas_saldo
, LAG(mcuentas_saldo, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mcuentas_saldo
, LAG(mautoservicio, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mautoservicio
, LAG(mautoservicio, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mautoservicio
, LAG(mprestamos_prendarios, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mprestamos_prendarios
, LAG(mprestamos_prendarios, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mprestamos_prendarios
, LAG(mprestamos_hipotecarios, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mprestamos_hipotecarios
, LAG(mprestamos_hipotecarios, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mprestamos_hipotecarios
, LAG(mplazo_fijo_dolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mplazo_fijo_dolares
, LAG(mplazo_fijo_dolares, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mplazo_fijo_dolares
, LAG(mplazo_fijo_pesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mplazo_fijo_pesos
, LAG(mplazo_fijo_pesos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mplazo_fijo_pesos
, LAG(minversion1_pesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_minversion1_pesos
, LAG(minversion1_pesos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_minversion1_pesos
, LAG(minversion1_dolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_minversion1_dolares
, LAG(minversion1_dolares, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_minversion1_dolares
, LAG(minversion2, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_minversion2
, LAG(minversion2, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_minversion2
, LAG(mpayroll, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mpayroll
, LAG(mpayroll, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mpayroll
, LAG(mpayroll2, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mpayroll2
, LAG(mpayroll2, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mpayroll2
, LAG(mcuenta_debitos_automaticos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mcuenta_debitos_automaticos
, LAG(mcuenta_debitos_automaticos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mcuenta_debitos_automaticos
, LAG(mttarjeta_visa_debitos_automaticos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mttarjeta_visa_debitos_automaticos
, LAG(mttarjeta_visa_debitos_automaticos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mttarjeta_visa_debitos_automaticos
, LAG(mttarjeta_master_debitos_automaticos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mttarjeta_master_debitos_automaticos
, LAG(mttarjeta_master_debitos_automaticos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mttarjeta_master_debitos_automaticos
, LAG(mpagomiscuentas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mpagomiscuentas
, LAG(mpagomiscuentas, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mpagomiscuentas
, LAG(mcajeros_propios_descuentos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mcajeros_propios_descuentos
, LAG(mcajeros_propios_descuentos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mcajeros_propios_descuentos
, LAG(mtarjeta_visa_descuentos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mtarjeta_visa_descuentos
, LAG(mtarjeta_visa_descuentos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mtarjeta_visa_descuentos
, LAG(mtarjeta_master_descuentos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mtarjeta_master_descuentos
, LAG(mtarjeta_master_descuentos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mtarjeta_master_descuentos
, LAG(mcomisiones_mantenimiento, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mcomisiones_mantenimiento
, LAG(mcomisiones_mantenimiento, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mcomisiones_mantenimiento
, LAG(mcomisiones_otras, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mcomisiones_otras
, LAG(mcomisiones_otras, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mcomisiones_otras
, LAG(mforex_buy, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mforex_buy
, LAG(mforex_buy, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mforex_buy
, LAG(mtransferencias_recibidas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mtransferencias_recibidas
, LAG(mtransferencias_recibidas, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mtransferencias_recibidas
, LAG(mtransferencias_emitidas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mtransferencias_emitidas
, LAG(mtransferencias_emitidas, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mtransferencias_emitidas
, LAG(mextraccion_autoservicio, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mextraccion_autoservicio
, LAG(mextraccion_autoservicio, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mextraccion_autoservicio
, LAG(mcheques_depositados, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mcheques_depositados
, LAG(mcheques_depositados, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mcheques_depositados
, LAG(mcheques_emitidos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mcheques_emitidos
, LAG(mcheques_emitidos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mcheques_emitidos
, LAG(mcheques_depositados_rechazados, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mcheques_depositados_rechazados
, LAG(mcheques_depositados_rechazados, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mcheques_depositados_rechazados
, LAG(mcheques_emitidos_rechazados, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mcheques_emitidos_rechazados
, LAG(mcheques_emitidos_rechazados, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mcheques_emitidos_rechazados
, LAG(matm, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_matm
, LAG(matm, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_matm
, LAG(matm_other, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_matm_other
, LAG(matm_other, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_matm_other
, LAG(mfinanciacion_limite_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mfinanciacion_limite_total
, LAG(mfinanciacion_limite_total, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mfinanciacion_limite_total
, LAG(msaldototal_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_msaldototal_total
, LAG(msaldototal_total, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_msaldototal_total
, LAG(msaldopesos_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_msaldopesos_total
, LAG(msaldopesos_total, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_msaldopesos_total
, LAG(msaldodolares_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_msaldodolares_total
, LAG(msaldodolares_total, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_msaldodolares_total
, LAG(mconsumospesos_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mconsumospesos_total
, LAG(mconsumospesos_total, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mconsumospesos_total
, LAG(mconsumosdolares_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mconsumosdolares_total
, LAG(mconsumosdolares_total, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mconsumosdolares_total
, LAG(mlimitecompra_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mlimitecompra_total
, LAG(mlimitecompra_total, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mlimitecompra_total
, LAG(madelantopesos_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_madelantopesos_total
, LAG(madelantopesos_total, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_madelantopesos_total
, LAG(madelantodolares_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_madelantodolares_total
, LAG(madelantodolares_total, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_madelantodolares_total
, LAG(mpagado_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mpagado_total
, LAG(mpagado_total, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mpagado_total
, LAG(mpagospesos_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mpagospesos_total
, LAG(mpagospesos_total, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mpagospesos_total
, LAG(mpagosdolares_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mpagosdolares_total
, LAG(mpagosdolares_total, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mpagosdolares_total
, LAG(mconsumototal_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mconsumototal_total
, LAG(mconsumototal_total, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mconsumototal_total
, LAG(mpagominimo_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mpagominimo_total
, LAG(mpagominimo_total, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mpagominimo_total
, LAG(mtarjetas_consumo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag1_mtarjetas_consumo
, LAG(mtarjetas_consumo, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag2_mtarjetas_consumo
    FROM competencia_03_DQ_agrupacion_var
),
cte_dif2 AS (
    SELECT
        *
        , dif1_mrentabilidad - LAG(dif1_mrentabilidad, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mrentabilidad
, dif1_mrentabilidad_annual - LAG(dif1_mrentabilidad_annual, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mrentabilidad_annual
, dif1_mcomisiones - LAG(dif1_mcomisiones, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mcomisiones
, dif1_mactivos_margen - LAG(dif1_mactivos_margen, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mactivos_margen
, dif1_mpasivos_margen - LAG(dif1_mpasivos_margen, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mpasivos_margen
, dif1_mcuenta_corriente - LAG(dif1_mcuenta_corriente, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mcuenta_corriente
, dif1_mcaja_ahorro - LAG(dif1_mcaja_ahorro, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mcaja_ahorro
, dif1_mcaja_ahorro_adicional - LAG(dif1_mcaja_ahorro_adicional, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mcaja_ahorro_adicional
, dif1_mcaja_ahorro_dolares - LAG(dif1_mcaja_ahorro_dolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mcaja_ahorro_dolares
, dif1_mcuentas_saldo - LAG(dif1_mcuentas_saldo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mcuentas_saldo
, dif1_mautoservicio - LAG(dif1_mautoservicio, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mautoservicio
, dif1_mprestamos_prendarios - LAG(dif1_mprestamos_prendarios, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mprestamos_prendarios
, dif1_mprestamos_hipotecarios - LAG(dif1_mprestamos_hipotecarios, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mprestamos_hipotecarios
, dif1_mplazo_fijo_dolares - LAG(dif1_mplazo_fijo_dolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mplazo_fijo_dolares
, dif1_mplazo_fijo_pesos - LAG(dif1_mplazo_fijo_pesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mplazo_fijo_pesos
, dif1_minversion1_pesos - LAG(dif1_minversion1_pesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_minversion1_pesos
, dif1_minversion1_dolares - LAG(dif1_minversion1_dolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_minversion1_dolares
, dif1_minversion2 - LAG(dif1_minversion2, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_minversion2
, dif1_mpayroll - LAG(dif1_mpayroll, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mpayroll
, dif1_mpayroll2 - LAG(dif1_mpayroll2, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mpayroll2
, dif1_mcuenta_debitos_automaticos - LAG(dif1_mcuenta_debitos_automaticos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mcuenta_debitos_automaticos
, dif1_mttarjeta_visa_debitos_automaticos - LAG(dif1_mttarjeta_visa_debitos_automaticos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mttarjeta_visa_debitos_automaticos
, dif1_mttarjeta_master_debitos_automaticos - LAG(dif1_mttarjeta_master_debitos_automaticos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mttarjeta_master_debitos_automaticos
, dif1_mpagomiscuentas - LAG(dif1_mpagomiscuentas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mpagomiscuentas
, dif1_mcajeros_propios_descuentos - LAG(dif1_mcajeros_propios_descuentos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mcajeros_propios_descuentos
, dif1_mtarjeta_visa_descuentos - LAG(dif1_mtarjeta_visa_descuentos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mtarjeta_visa_descuentos
, dif1_mtarjeta_master_descuentos - LAG(dif1_mtarjeta_master_descuentos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mtarjeta_master_descuentos
, dif1_mcomisiones_mantenimiento - LAG(dif1_mcomisiones_mantenimiento, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mcomisiones_mantenimiento
, dif1_mcomisiones_otras - LAG(dif1_mcomisiones_otras, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mcomisiones_otras
, dif1_mforex_buy - LAG(dif1_mforex_buy, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mforex_buy
, dif1_mtransferencias_recibidas - LAG(dif1_mtransferencias_recibidas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mtransferencias_recibidas
, dif1_mtransferencias_emitidas - LAG(dif1_mtransferencias_emitidas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mtransferencias_emitidas
, dif1_mextraccion_autoservicio - LAG(dif1_mextraccion_autoservicio, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mextraccion_autoservicio
, dif1_mcheques_depositados - LAG(dif1_mcheques_depositados, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mcheques_depositados
, dif1_mcheques_emitidos - LAG(dif1_mcheques_emitidos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mcheques_emitidos
, dif1_mcheques_depositados_rechazados - LAG(dif1_mcheques_depositados_rechazados, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mcheques_depositados_rechazados
, dif1_mcheques_emitidos_rechazados - LAG(dif1_mcheques_emitidos_rechazados, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mcheques_emitidos_rechazados
, dif1_matm - LAG(dif1_matm, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_matm
, dif1_matm_other - LAG(dif1_matm_other, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_matm_other
, dif1_mfinanciacion_limite_total - LAG(dif1_mfinanciacion_limite_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mfinanciacion_limite_total
, dif1_msaldototal_total - LAG(dif1_msaldototal_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_msaldototal_total
, dif1_msaldopesos_total - LAG(dif1_msaldopesos_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_msaldopesos_total
, dif1_msaldodolares_total - LAG(dif1_msaldodolares_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_msaldodolares_total
, dif1_mconsumospesos_total - LAG(dif1_mconsumospesos_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mconsumospesos_total
, dif1_mconsumosdolares_total - LAG(dif1_mconsumosdolares_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mconsumosdolares_total
, dif1_mlimitecompra_total - LAG(dif1_mlimitecompra_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mlimitecompra_total
, dif1_madelantopesos_total - LAG(dif1_madelantopesos_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_madelantopesos_total
, dif1_madelantodolares_total - LAG(dif1_madelantodolares_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_madelantodolares_total
, dif1_mpagado_total - LAG(dif1_mpagado_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mpagado_total
, dif1_mpagospesos_total - LAG(dif1_mpagospesos_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mpagospesos_total
, dif1_mpagosdolares_total - LAG(dif1_mpagosdolares_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mpagosdolares_total
, dif1_mconsumototal_total - LAG(dif1_mconsumototal_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mconsumototal_total
, dif1_mpagominimo_total - LAG(dif1_mpagominimo_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mpagominimo_total
, dif1_mtarjetas_consumo - LAG(dif1_mtarjetas_consumo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mtarjetas_consumo
    FROM cte_dif1
)
SELECT *
FROM cte_dif2;

In [12]:
%%sql
SELECT COUNT(*) AS num_columnas
FROM pragma_table_info('competencia_03_DQ_ft');

,num_columnas
0,288


In [9]:
dataset_output= dataset_path + 'competencia_03_DQ_ft_2.parquet'

In [10]:
%%sql
ROLLBACK;

,Success


In [11]:
%%sql
COPY competencia_03_DQ_ft TO '{{dataset_output}}' (FORMAT PARQUET);

,Success
